In [4]:
%matplotlib ipympl
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import gsw
from glob import glob

In [5]:
temp_nc = 'https://data.nodc.noaa.gov/thredds/dodsC/nodc/archive/data/0114815/public/temperature/netcdf/decav/1.00/woa13_decav_t00_01.nc'
sal_nc = 'https://data.nodc.noaa.gov/thredds/dodsC/nodc/archive/data/0114815/public/salinity/netcdf/decav/1.00/woa13_decav_s00_01.nc'

In [6]:
temp = xr.open_dataset(temp_nc,decode_times=False)
sal = xr.open_dataset(sal_nc,decode_times=False)

In [7]:
# position of ADCP 
mlat = 35+8.4585/60  
mlon = -122-59.9036/60

In [8]:
f = gsw.f(mlat)

In [9]:
ii = np.array(np.argmin(np.abs(mlat-temp['lat'])))
jj = np.array(np.argmin(np.abs(mlon-temp['lon'])))

In [10]:
mtemp_an = np.squeeze(temp['t_an'][0,:,ii,jj])
msal_an = np.squeeze(sal['s_an'][0,:,ii,jj])
mtemp_mn = np.squeeze(temp['t_mn'][0,:,ii,jj])
msal_mn = np.squeeze(sal['s_mn'][0,:,ii,jj])
mz = np.array(temp['depth'])

In [11]:
mpr = gsw.p_from_z(-mz,temp['lat'][ii])
mSA_mn = msal_mn
mCT_mn = gsw.CT_from_t(mSA_mn,mtemp_mn,mpr)
msigma0 = gsw.sigma0(mSA_mn,mCT_mn)
msigma4 = gsw.sigma4(mSA_mn,mCT_mn)

mNsq,mpr_mid = gsw.Nsquared(mSA_mn,mCT_mn,mpr,lat=temp['lat'][ii])
mN = np.sqrt(mNsq)

In [14]:
dflist = []

plt.figure()
ax1 = plt.subplot(121)
ax2 = plt.subplot(122)

hwod, = ax1.plot(msigma4,mpr,'k-',lw=2)
ax1.invert_yaxis()
#ax1.legend(hwod,('World Ocean Atlas'))
ax1.set_ylabel('pressure [dbar]')
ax1.set_xlabel('$\sigma_4$ [kg/m$^3$]')
ax1.set_title('potential density\n4000 dbar reference')
xl = ax1.get_xlim()
yl = ax1.get_ylim()
ax1.set_ylim([yl[0],0])
ax1.text(xl[0]+0.04*np.diff(xl),yl[1]-0.1*np.diff(yl),'a)')

ax2.semilogx(mN,mpr_mid,'k-',lw=2)
ax2.invert_yaxis()
ax2.get_yaxis().set_ticklabels([])
ax2.set_title('buoyancy frequency')
ax2.set_xlabel('$N$ [rad s$^{-1}$]')
xl = ax2.get_xlim()
yl = ax2.get_ylim()
ax2.set_ylim([yl[0],0])
ax2.text(xl[0]+0.001*np.diff(xl),yl[1]-0.1*np.diff(yl),'b)')
plt.tight_layout()
plt.savefig('figures_paper/water_column_strat.pdf')
plt.savefig('figures_paper/water_column_strat.png')

FigureCanvasNbAgg()